In [ ]:

import math
from Core.PRef import PRef
import numpy as np


class SPRef:
    sessions: list[set[int]]
    fitnesses: np.ndarray
    
    def __init__(self,
                 sessions: list[set[int]],
                 fitnesses: np.ndarray):
        self.sessions = sessions
        self.fitnesses = fitnesses
        
        
    @classmethod
    def from_pRef(cls, pRef: PRef):
        def row_to_set(row):
            return {index for index, value in row if value}
        return cls(list(map(row_to_set, pRef.full_solution_matrix)), pRef.fitness_array)

    
    
    def get_matching_fitnesses(self, ps: set[int], threshold: float):
        most_leftover = math.floor(len(ps) * (1-threshold))
        fitnesses = []
        for session, fitness in zip(self.sessions, self.fitnesses):
            if len(ps-session) < most_leftover:
                fitnesses.append(fitness)
        return np.array(fitnesses)
    
    


In [ ]:
def simplicity(ps):
    return len(ps)

def make_mean_fitness(sPRef: SPRef, threshold: float = 0.5):
    global_average = np.average(sPRef.fitnesses)
    terr
    def mean_fitness(ps):
        if ps.is_empty():
            return -global_average
        match_fitnesses = sPRef.get_matching_fitnesses(ps, threshold=0.5)
        if match_fitnesses:
            return -np.average(match_fitnesses)
        else:
            return 